In [2]:
import os
import pandas as pd
import numpy as np

In [24]:
to_correct_ids = ['76_1', '76_2', '76_3', '77_1', '77_2', '77_3', '78_1', '78_2', 
                   '78_3', '79_1', '79_2', '79_3', '80_1', '80_2', '80_3', '81_1', 
                   '81_2', '81_3', '82_1', '82_2', '82_3', '83_1', '83_2', '83_3', 
                   '84_1', '84_2', '84_3', '85_1', '85_2', '85_3', '86_1', '86_2', 
                   '86_3', '87_1', '87_2', '87_3', '88_1', '88_2', '88_3', '89_1', 
                   '89_2', '89_3', '90_1', '90_2', '90_3', '91_1', '91_2', '91_3', 
                   '92_1', '92_2', '92_3', '93_1', '93_2', '93_3', '94_1', '94_2', 
                   '94_3', '95_1', '95_2', '95_3', '96_1', '96_2', '96_3', '97_1', 
                   '97_2', '97_3', '98_1', '98_2', '98_3', '99_1', '99_2', '99_3', 
                   '100_1', '100_2', '100_3']

to_correct_sent_ids = list(set([i.split('_')[0] for i in to_correct_ids]))
#to_correct_sent_ids

In [25]:
def get_participant_model_column(full_folder_path, participant_id, model):

    fname_list = []
    transcript_list = []
    participant_id_list = []
    sentence_ids_list = []
    iterations_list = []
    
    for file in os.listdir(full_folder_path):
        if 'whisper' not in file and file.endswith('.txt') and model in file:

            #print(file.split('_')[6])
            
            # for this subset of files found
            # adding to check for F or M in t2_F or t2_M
            # after checking t2, t2_F wav files are blank
            # only t2_M files contain content we want
            f_or_m = file.split('_')[1]

            if (file.split('_')[6] in to_correct_sent_ids and f_or_m == "M") or file.split('_')[6] not in to_correct_sent_ids: 
                #looking through only the M files for relevant ids, all files for others
            
                fname_list.append(file)
                
                full_fname = os.path.join(full_folder_path, file)
                                
                ids = file.split('_')[6:9]
    
                sentence_id = ids[0] + '_' + ids[1]
                sentence_ids_list.append(sentence_id)
    
                iteration = ids[2]
                iterations_list.append(iteration)
                
                with open(full_fname, 'r') as f:
                    transcript = f.read()
                    transcript = transcript.strip()
                    #print(transcript)
                    transcript_list.append(transcript)
    
                participant_id_list.append(participant_id)

    result_df = pd.DataFrame({'participant_id':participant_id_list})

    result_df['file_name'] = fname_list # kept for debugging

    result_df['Sentence ID'] = sentence_ids_list

    result_df['Iteration'] = iterations_list

    result_df[model] = transcript_list

    return result_df

In [26]:
'''test_df = get_participant_model_column('/Users/oishanibandopadhyay/Documents/LASR_Whisper_SAUCE/sentence_transcription/t2_sauce', 
                             't2',
                            'large')
test_df'''

"test_df = get_participant_model_column('/Users/oishanibandopadhyay/Documents/LASR_Whisper_SAUCE/sentence_transcription/t2_sauce', \n                             't2',\n                            'large')\ntest_df"

In [28]:
def merging_models_per_participant(full_folder_path, participant_id):
    df_merged = get_participant_model_column(
    full_folder_path, participant_id, 'large')
    
    model_list = ['base', 'medium', 'small', 'tiny']
    for model in model_list:
        model_df = get_participant_model_column(
            full_folder_path, participant_id, model)
        
        df_merged = df_merged.merge(
            model_df[['participant_id', 'Sentence ID', 'Iteration', model]],
            on = ['participant_id', 'Sentence ID', 'Iteration'],
            how = 'inner')

    return df_merged
    

In [29]:
participant_df = merging_models_per_participant('/Users/oishanibandopadhyay/Documents/LASR_Whisper_SAUCE/sentence_transcription/t2_sauce', 
                             't2')

In [30]:
def merge_with_targets(participant_df):
    target_sentence_df = pd.read_csv('target_sentences.csv')
    
    merged_df = participant_df.merge(
        target_sentence_df, on = 'Sentence ID', how = 'left')

    merged_df = merged_df[['participant_id', 'file_name', 'Sentence ID', 
                           'Iteration', 'probability', 'Expected Transcription',
                           'large', 'base', 'medium', 
                           'small', 'tiny']].reindex()
    return merged_df
    

In [31]:
merge_with_targets(participant_df)

,participant_id,file_name,Sentence ID,Iteration,probability,Expected Transcription,large,base,medium,small,tiny
0,t2,t2_F_MN_SAUCE_block0_sauce_23_2_L1_large-v3.txt,23_2,L1,high prob,When daddy cooks he adds pepper and salt.,"When daddy cooks, he adds pepper and salt.","When daddy clicks, he adds pepper and salt.",when daddy cooks he adds pepper and salt.,"When daddy cooks, he adds pepper and salt.","When Daddy Clues, he adds pepper and salt."
1,t2,t2_F_MN_SAUCE_block0_sauce_9_2_L2_large-v3.txt,9_2,L2,nonword,I put shamTROO in my pig.,I push them through in my peg.,I will lace in my back.,I push them true in my pig.,I push them through in my pick.,I will show them true in my pack.
2,t2,t2_F_MN_SAUCE_block0_sauce_20_1_L2_large-v3.txt,20_1,L2,low prob,Please sweep the floor with a snowman.,Please sweep the floor with a snowman.,Place with the floor with a snowman.,Place with the floor with a snowman.,Place with the floor with a snowman.,Place with the floor with a snowman.
3,t2,t2_F_MN_SAUCE_block0_sauce_31_3_L2_large-v3.txt,31_3,L2,low prob,The pilot flew the rope.,The pilot flew the rope.,The pilot flew the rope.,the pile of lute rope.,The pilot fluid rope.,The pilot flew the rope.
4,t2,t2_F_MN_SAUCE_block0_sauce_54_2_L2_large-v3.txt,54_2,L2,nonword,"In the plathtub, there is a floating little wh...","In the flat top, there is a floating little wh...",In the flat top there's a floating little wheel.,"In the flat top, there's a floating little wheel.","In the flat top, there's a floating little wheel.",In the flat top there's a floating little wheel.
...,...,...,...,...,...,...,...,...,...,...,...
637,t2,t2_M_MN_SAUCE_block4_sauce_81_2_L2_large-v3.txt,81_2,L2,high prob,In the pond there was one quacking duck.,In the pound there was one quacking dog.,"In the pound, you were squawking dog.","In the pound, there was one quacking dog.","In the pound, there was one quacking dog.","In the pound, you are in a quacking dog."
638,t2,t2_F_MN_SAUCE_block0_sauce_27_2_L2_large-v3.txt,27_2,L2,nonword,The fimmer was being chased by a giant house.,The femur was being chased by a giant house.,The femur was being chased by a giant house.,The femur was being chased by a giant house.,The femur was being chased by a giant house.,The Fimer was being chased by a giant house.
639,t2,t2_F_MN_SAUCE_block0_sauce_35_3_L1_large-v3.txt,35_3,L1,high prob,"At the haunted house, I saw a ghost.","At the haunted house, I saw a ghost.",At the haunted house I saw it ghost.,"At the haunted house, I saw a ghost.","At the haunted house, I saw a ghost.","At the haunted house, I saw it ghost."
640,t2,t2_F_MN_SAUCE_block0_sauce_69_2_L2_large-v3.txt,69_2,L2,nonword,Spids sit in the back seat of the bee.,Spitz sit in the back seat of the bee.,Speed set in the back seat of the V.,Spits sit in the back seat of the VE.,Spit sit in the back seat of the bee.,Spits sit in the back seat of the bee.


In [33]:
def full_txt_to_df(full_sauce_folder_path, participant_id, destination_path):
    participant_df = merging_models_per_participant(
        full_sauce_folder_path, participant_id)

    full_df = merge_with_targets(participant_df)
    
    full_df.to_csv(destination_path)

    return full_df

In [34]:
full_txt_to_df('/Users/oishanibandopadhyay/Documents/LASR_Whisper_SAUCE/sentence_transcription/t2_sauce', 
               't2', 
               '/Users/oishanibandopadhyay/Documents/LASR_Whisper_SAUCE/sentence_transcription/t2_sauce/t2_sauce_corrected.csv')


,participant_id,file_name,Sentence ID,Iteration,probability,Expected Transcription,large,base,medium,small,tiny
0,t2,t2_F_MN_SAUCE_block0_sauce_23_2_L1_large-v3.txt,23_2,L1,high prob,When daddy cooks he adds pepper and salt.,"When daddy cooks, he adds pepper and salt.","When daddy clicks, he adds pepper and salt.",when daddy cooks he adds pepper and salt.,"When daddy cooks, he adds pepper and salt.","When Daddy Clues, he adds pepper and salt."
1,t2,t2_F_MN_SAUCE_block0_sauce_9_2_L2_large-v3.txt,9_2,L2,nonword,I put shamTROO in my pig.,I push them through in my peg.,I will lace in my back.,I push them true in my pig.,I push them through in my pick.,I will show them true in my pack.
2,t2,t2_F_MN_SAUCE_block0_sauce_20_1_L2_large-v3.txt,20_1,L2,low prob,Please sweep the floor with a snowman.,Please sweep the floor with a snowman.,Place with the floor with a snowman.,Place with the floor with a snowman.,Place with the floor with a snowman.,Place with the floor with a snowman.
3,t2,t2_F_MN_SAUCE_block0_sauce_31_3_L2_large-v3.txt,31_3,L2,low prob,The pilot flew the rope.,The pilot flew the rope.,The pilot flew the rope.,the pile of lute rope.,The pilot fluid rope.,The pilot flew the rope.
4,t2,t2_F_MN_SAUCE_block0_sauce_54_2_L2_large-v3.txt,54_2,L2,nonword,"In the plathtub, there is a floating little wh...","In the flat top, there is a floating little wh...",In the flat top there's a floating little wheel.,"In the flat top, there's a floating little wheel.","In the flat top, there's a floating little wheel.",In the flat top there's a floating little wheel.
...,...,...,...,...,...,...,...,...,...,...,...
637,t2,t2_M_MN_SAUCE_block4_sauce_81_2_L2_large-v3.txt,81_2,L2,high prob,In the pond there was one quacking duck.,In the pound there was one quacking dog.,"In the pound, you were squawking dog.","In the pound, there was one quacking dog.","In the pound, there was one quacking dog.","In the pound, you are in a quacking dog."
638,t2,t2_F_MN_SAUCE_block0_sauce_27_2_L2_large-v3.txt,27_2,L2,nonword,The fimmer was being chased by a giant house.,The femur was being chased by a giant house.,The femur was being chased by a giant house.,The femur was being chased by a giant house.,The femur was being chased by a giant house.,The Fimer was being chased by a giant house.
639,t2,t2_F_MN_SAUCE_block0_sauce_35_3_L1_large-v3.txt,35_3,L1,high prob,"At the haunted house, I saw a ghost.","At the haunted house, I saw a ghost.",At the haunted house I saw it ghost.,"At the haunted house, I saw a ghost.","At the haunted house, I saw a ghost.","At the haunted house, I saw it ghost."
640,t2,t2_F_MN_SAUCE_block0_sauce_69_2_L2_large-v3.txt,69_2,L2,nonword,Spids sit in the back seat of the bee.,Spitz sit in the back seat of the bee.,Speed set in the back seat of the V.,Spits sit in the back seat of the VE.,Spit sit in the back seat of the bee.,Spits sit in the back seat of the bee.
